# DFS version
## Basic idea

The basic idea of this method is to fit every pieces within a square block
```python
board = [[0,0],
         [-1,0]]
block = [[0,0],
         [1,1]]
```
For block 1 represent a block, 0 represent emptiness

For board -1 means the hole, 1~9 means the block_id, 0 means empty

In the example above the block can't fit into board, as left bottom coincide.

Like filtering a image, we use the window of block to try to fit every pieces into every part of the board. Whenever failed we rotate the block and try again. After all position have been tried and failed, return False. If every block have been fit into the board return True.

## Drawback
The problem of this method is 
```python
 [[1,0,0,0,0],
  [1,0,0,0,0],
  [1,0,0,0,0],
  [1,0,0,0,0],
  [1,0,0,0,0]]
```
and
```python
 [[0,1,0,0,0],
  [0,1,0,0,0],
  [0,1,0,0,0],
  [0,1,0,0,0],
  [0,1,0,0,0]]
```
are different block.

Also the time complexity is too large.

# Todo & Thoughts

1. Add isolated 0 detaction to trim the search tree
1. Add a heuristic and try A* instead of DFS

# Code

In [ ]:
import numpy as np
# one way to do the rotation
def rotate90_counterclockwise(matrix):
    for i in range(len(matrix)//2):
        for j in range(len(matrix)):
            tmp = matrix[j][i]
            matrix[j][i] = matrix[j][len(matrix)-i-1]
            matrix[j][len(matrix)-i-1] = tmp
    for i in range(1,len(matrix)):
        for j in range(i):
            tmp=matrix[i][j]
            matrix[i][j]=matrix[j][i]
            matrix[j][i]=tmp
    return matrix


# board is a part of board
# block is of the same size of board
# Example: board = [[ 0,0],
#                   [-1,0]]
#          block = [[0,0],
#                   [1,1]]
# For blocks 1 represent a block, 0 represent emptiness
# For board -1 means the hole, 1~9 means the square of blocks, 0 means empty
# Test wether a piece can fit into certain part of board
# Board and block have to be same size and both square
def is_fit(board, block):
    for x in range(board.shape[0]):
        for y in range(board.shape[0]):
            if block[x][y] == 1:
                if board[x][y] == 0:
                    continue
                else:
                    return False
    return True

# put one piece on board
def put_fit(board, block, block_id):
    for x in range(len(block)):
        for y in range(len(block[0])):
            if block[x][y] == 1:
                board[x][y] = block_id

# remove one piece on board
def unput_fit(board, block):
    for x in range(len(block)):
        for y in range(len(block[0])):
            if block[x][y] == 1:
                board[x][y] = 0

# Rotate the block four times and try to fit it into board
# It's not a good solution to use just dfs
def solution(board, holes, blocks):        
    def dfs(board, block_id):
        if block_id == len(blocks)+1:
            return True
        block = blocks[block_id - 1]
        N = len(block)
        # Move the window of block to every position of board and try to fit in
        for x in range(len(board) - N + 1):
            for y in range(len(board) - N + 1):
                block = blocks[block_id - 1]
                r = 0
                while r < 4: # four direction
                    if is_fit(board[x:x + N, y:y + N], block):
                        put_fit(board[x:x + N, y:y + N], block, block_id)
                        # Just try get one solution first
                        if dfs(board, block_id + 1):
                            return True
                        else:
                            # failed, remove the block from board
                            unput_fit(board[x:x + N, y:y + N], block)
                            block = rotate90_counterclockwise(block)
                            r += 1
                    else:
                        block = rotate90_counterclockwise(block)
                        r += 1
        return False
    
    board_tmp = board.copy()
    # for each hole on the board
    for i in range(len(holes)):
        board = board_tmp.copy()
        board[holes[i][0]][holes[i][1]] = -1
        print(dfs(board,1))
        print(board)



# Test

In [ ]:
board = np.zeros((3,3))
holes = [[2,0],[0,0],[0,1],[0,2]]
blocks = [[[1,0],  #0
           [1,0]],
          [[1,1],  #1
           [1,0]], 
          [[1,1],  #2
           [1,0]]
         ]
solution(board,holes,blocks)

In [ ]:
# The time complexity is too large and This one runs forever
board = np.zeros((7,7))
# holes = [[0,3], [1,0], [1,4], [1,5], [3,2], [3,3], [4,1], [4,4], [4,6], [5,3], [6,0], [6,4], [6,5]]
holes = [[6,0]]
# blocks = [[[1,0],
#           [1,0]],
#          [[1,0,0,0,0], #2
#           [1,0,0,0,0],
#           [1,0,0,0,0],
#           [1,0,0,0,0],
#           [1,0,0,0,0]],
#          [[0,1,0,0,0],
#           [0,1,0,0,0],
#           [0,1,0,0,0],
#           [0,1,0,0,0],
#           [0,1,0,0,0]],
#          [[0,0,1,0,0],
#           [0,0,1,0,0],
#           [0,0,1,0,0],
#           [0,0,1,0,0],
#           [0,0,1,0,0]],
#          [[0,1,1,0], #3
#           [1,1,1,0],
#           [0,1,1,1],
#           [0,0,0,0]],
#          [[0,0,0,0],
#           [0,1,1,0],
#           [1,1,1,0],
#           [0,1,1,1]],
#          [[1,1,0], #4
#           [0,1,1],
#           [0,0,1]],
#          [[1,1,0], #5
#           [1,1,1],
#           [1,1,0]],
#          [[0,1,0,0], #6
#           [1,1,0,0],
#           [1,1,0,0],
#           [0,1,0,0]],
#          [[0,0,1,0],
#           [0,1,1,0],
#           [0,1,1,0],
#           [0,0,1,0]],
#          [[1,0,0], #7
#           [1,1,0],
#           [1,1,1]],
#          [[0,0,0], #8
#           [0,1,0],
#           [1,1,1]],
#          [[0,1,0],
#           [1,1,1],
#           [0,0,0]],
#          [[1,1,1,1], #9
#           [0,0,1,0],
#           [0,0,0,0],
#           [0,0,0,0]],
#          [[0,0,0,0],
#           [1,1,1,1],
#           [0,0,1,0],
#           [0,0,0,0]]
#         ]

# reducted version of 7*7
blocks = [[[1,0],
          [1,0]],
         [[1,0,0,0,0], #2
          [1,0,0,0,0],
          [1,0,0,0,0],
          [1,0,0,0,0],
          [1,0,0,0,0]],
         [[0,1,1,0], #3
          [1,1,1,0],
          [0,1,1,1],
          [0,0,0,0]],
         [[1,1,0], #4
          [0,1,1],
          [0,0,1]],
         [[1,1,0], #5
          [1,1,1],
          [1,1,0]],
         [[0,1,0,0], #6
          [1,1,0,0],
          [1,1,0,0],
          [0,1,0,0]],
         [[1,0,0], #7
          [1,1,0],
          [1,1,1]],
         [[0,0,0], #8
          [0,1,0],
          [1,1,1]],
         [[1,1,1,1], #9
          [0,0,1,0],
          [0,0,0,0],
          [0,0,0,0]]
        ]
solution(board,holes,blocks)

In [ ]:
# 4*4 easy problem
board = np.zeros((4,4))
holes = [[3,0]]
blocks = [[[1,1],  #0
           [1,0]],
          [[1,1,0],
           [0,1,1],
           [0,0,1]],
          [[1,0,0],
           [1,1,0],
           [0,1,0]],
          [[1,1],  #2
           [1,0]]
         ]
solution(board,holes,blocks)

# Using OOP and state space representation

A state of this problem contains:
1. board: the state of board with 1~7 represent different block
2. block_id: how many pieces have been placed

Call `state.next_level()` to generate a list of node with next block placed on board

In [40]:
import numpy as np
class State:
    # class variables of the puzzle
    holes = None
    blocks = None
    # block_id represent all blocks before the id have been placed on board
    def __init__(self, board = None, block_id = 0):
        self.board = board
        self.block_id = block_id
    def put_hole(self,hole_id):
        self.board[State.holes[hole_id][0]][State.holes[hole_id][1]] = -1
    # staic method
    def is_fit(board, block):
        for x in range(board.shape[0]):
            for y in range(board.shape[0]):
                if block[x][y] == 1:
                    if board[x][y] == 0:
                        continue
                    else:
                        return False
        return True
    
    def rotate90_counterclockwise(matrix):
        for i in range(len(matrix)//2):
            for j in range(len(matrix)):
                tmp = matrix[j][i]
                matrix[j][i] = matrix[j][len(matrix)-i-1]
                matrix[j][len(matrix)-i-1] = tmp
        for i in range(1,len(matrix)):
            for j in range(i):
                tmp=matrix[i][j]
                matrix[i][j]=matrix[j][i]
                matrix[j][i]=tmp
        return matrix
    
    # put one piece on board
    def put_fit(board, block, block_id):
        for x in range(len(block)):
            for y in range(len(block[0])):
                if block[x][y] == 1:
                    board[x][y] = block_id
    def unput_fit(board, block):
        for x in range(len(block)):
            for y in range(len(block[0])):
                if block[x][y] == 1:
                    board[x][y] = 0
                    
    def print_next_level(self):
        board_list = self.next_level()
        for i in range(len(board_list)):
            print(str(i) + ": block_id = " + str(board_list[i].block_id))
            print(board_list[i].board)
    # return a list of possible state from current state
    # return True if all blocks have been put on the board
    # puts next blocks on the board
    def next_level(self):
        self.block_id += 1
        board_list = []
        if self.block_id == len(State.blocks) + 1:
            return True
        block = State.blocks[self.block_id - 1]
        N = len(block)
        for x in range(len(self.board) - N + 1):
            for y in range(len(self.board) - N + 1):
                block = State.blocks[self.block_id - 1]
                r = 0
                while r < 4:  # four direction
                    if State.is_fit(self.board[x:x + N, y:y + N], block):
                        State.put_fit(self.board[x:x + N, y:y + N], block, self.block_id)
                        next_state = State(self.board.copy(),self.block_id)
                        board_list.append(next_state)
                        State.unput_fit(self.board[x:x + N, y:y + N], block)
                        block = State.rotate90_counterclockwise(block)
                        r += 1
                    else:
                        block = State.rotate90_counterclockwise(block)
                        r += 1
        return board_list

        

In [41]:
State.holes = [[3,0]]
State.blocks =   [[[1,1],  #0
                   [1,0]],
                  [[1,1,0],
                   [0,1,1],
                   [0,0,1]],
                  [[1,0,0],
                   [1,1,0],
                   [0,1,0]],
                  [[1,1],  #2
                   [1,0]]]
s1 = State(np.zeros((4,4)),0)
s1.put_hole(0)
board_list = s1.next_level()

In [38]:
for i in range(len(board_list)):
    print(str(i) + ": block_id = " + str(board_list[i].block_id))
    print(board_list[i].board)


0: block_id = 1
[[ 1.  1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [-1.  0.  0.  0.]]
1: block_id = 1
[[ 1.  0.  0.  0.]
 [ 1.  1.  0.  0.]
 [ 0.  0.  0.  0.]
 [-1.  0.  0.  0.]]
2: block_id = 1
[[ 0.  1.  0.  0.]
 [ 1.  1.  0.  0.]
 [ 0.  0.  0.  0.]
 [-1.  0.  0.  0.]]
3: block_id = 1
[[ 1.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  0.]
 [-1.  0.  0.  0.]]
4: block_id = 1
[[ 0.  1.  1.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  0.]
 [-1.  0.  0.  0.]]
5: block_id = 1
[[ 0.  1.  0.  0.]
 [ 0.  1.  1.  0.]
 [ 0.  0.  0.  0.]
 [-1.  0.  0.  0.]]
6: block_id = 1
[[ 0.  0.  1.  0.]
 [ 0.  1.  1.  0.]
 [ 0.  0.  0.  0.]
 [-1.  0.  0.  0.]]
7: block_id = 1
[[ 0.  1.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  0.]
 [-1.  0.  0.  0.]]
8: block_id = 1
[[ 0.  0.  1.  1.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0.  0.]
 [-1.  0.  0.  0.]]
9: block_id = 1
[[ 0.  0.  1.  0.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  0.  0.]
 [-1.  0.  0.  0.]]
10: block_id = 1
[[ 0.  0.  0.  1.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  0.  

In [42]:
board_list[0].print_next_level()

0: block_id = 2
[[ 1.  1.  2.  0.]
 [ 1.  2.  2.  0.]
 [ 2.  2.  0.  0.]
 [-1.  0.  0.  0.]]
1: block_id = 2
[[ 1.  1.  2.  2.]
 [ 1.  2.  2.  0.]
 [ 0.  2.  0.  0.]
 [-1.  0.  0.  0.]]
2: block_id = 2
[[ 1.  1.  0.  2.]
 [ 1.  0.  2.  2.]
 [ 0.  2.  2.  0.]
 [-1.  0.  0.  0.]]
3: block_id = 2
[[ 1.  1.  0.  0.]
 [ 1.  2.  2.  0.]
 [ 0.  0.  2.  2.]
 [-1.  0.  0.  2.]]
4: block_id = 2
[[ 1.  1.  0.  0.]
 [ 1.  0.  2.  2.]
 [ 0.  2.  2.  0.]
 [-1.  2.  0.  0.]]
5: block_id = 2
[[ 1.  1.  0.  0.]
 [ 1.  2.  0.  0.]
 [ 0.  2.  2.  0.]
 [-1.  0.  2.  2.]]
6: block_id = 2
[[ 1.  1.  0.  0.]
 [ 1.  0.  0.  2.]
 [ 0.  0.  2.  2.]
 [-1.  2.  2.  0.]]
